In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

In [2]:
from modules.data.conll2003.prc import conll2003_preprocess

In [12]:
data_dir = "/home/eartemov/ae/work/conll2003/"

In [13]:
conll2003_preprocess(data_dir)

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels5.txt",
    clear_cache=True,
    model_name="bert-base-cased"
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [4]:
from modules.models.bert_models import BERTBiLSTMAttnNCRF

In [5]:
model = BERTBiLSTMAttnNCRF.create(
    len(data.train_ds.idx2label), model_name="bert-base-cased",
    lstm_dropout=0., crf_dropout=0.3, nbest=len(data.train_ds.idx2label))

build CRF...


In [6]:
from modules.train.train import NerLearner

In [7]:
num_epochs = 100

In [8]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMAttnNCRF-base-IO.cpt",
    t_total=num_epochs * len(data.train_dl))

In [9]:
model.get_n_trainable_params()

2630289

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [21]:
from modules.data.bert_data import get_data_loader_for_predict

In [22]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [23]:
preds = learner.predict(dl)

In [24]:
from sklearn_crfsuite.metrics import flat_classification_report

In [25]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [31]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [32]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [33]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.9552    0.9461    0.9506      2005
         I_O     0.9966    0.9973    0.9970     41651
      I_MISC     0.9343    0.8956    0.9146      1255
       I_PER     0.9846    0.9863    0.9854      2848
       I_LOC     0.9727    0.9636    0.9681      1922

   micro avg     0.9918    0.9907    0.9913     49681
   macro avg     0.9687    0.9578    0.9631     49681
weighted avg     0.9917    0.9907    0.9912     49681



### Test

In [11]:
from modules.data.bert_data import get_data_loader_for_predict

In [12]:
dl = get_data_loader_for_predict(data, df_path="/home/eartemov/ae/work/conll2003/eng.testb.dev.csv")

In [13]:
preds = learner.predict(dl)

In [14]:
from sklearn_crfsuite.metrics import flat_classification_report

In [15]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [16]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [17]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [18]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8891    0.9174    0.9030      2360
         I_O     0.9952    0.9910    0.9931     37492
      I_MISC     0.8519    0.8154    0.8332       910
       I_PER     0.9729    0.9754    0.9741      2683
       I_LOC     0.9217    0.9207    0.9212      1815

   micro avg     0.9824    0.9798    0.9811     45260
   macro avg     0.9261    0.9240    0.9249     45260
weighted avg     0.9825    0.9798    0.9811     45260

